In [1]:
import pandas as pd
import statistics

In [2]:
ticker_name="MSFT" #ONLY THING TO CHANGE
stock_daily=pd.read_csv(f"HistoricalData/{ticker_name}_DailyAdjusted.csv")
stock_daily["HLOC_AV"]=(stock_daily["Open"]+stock_daily["High"]+stock_daily["Low"]+stock_daily["Close"])/4
stock_daily.head()

,Date,Open,High,Low,Close,Volume,AdjustedClose,HLOC_AV
0,2022-08-19,288.90,289.25,285.56,286.15,20557200,286.1500,287.4650
1,2022-08-18,290.19,291.91,289.08,290.17,17186200,290.1700,290.3375
2,2022-08-17,289.74,293.35,289.47,291.32,18253400,291.3200,290.9700
3,2022-08-16,291.99,294.04,290.42,292.71,18102900,292.0900,292.2900
4,2022-08-15,291.00,294.18,290.11,293.47,18085700,292.8484,292.1900


In [3]:
for i in range(10,40):
    stock_daily["T-"+str(i)+"_HLOC_AV"]=None
    stock_daily["T-"+str(i)+"_Volume"]=None
    stock_daily["T-"+str(i)+"_PChange"]=None
    if i%3==0:
        stock_daily[str(i-9)+"MA"]=None
stock_daily=stock_daily.copy()

for k in range(3,30,3):
    stock_daily[str(k)+"EMA"]=None
stock_daily=stock_daily.copy()

for k in range(3,22,3):
    stock_daily[str(k+1)+"_StandardDev"]=None

stock_daily["RSI"]=None
stock_daily["OnBalanceVolume"]=None
stock_daily["StochasticOsc"]=None
stock_daily["BollingerU"]=None
stock_daily["BollingerL"]=None
stock_daily["MACD"]=None
stock_daily=stock_daily.copy()

stock_daily.drop(labels="T-10_PChange", axis=1, inplace=True)

C:\Users\baris\AppData\Local\Temp\ipykernel_1344\2265578016.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_daily["T-"+str(i)+"_HLOC_AV"]=None
C:\Users\baris\AppData\Local\Temp\ipykernel_1344\2265578016.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_daily["T-"+str(i)+"_Volume"]=None
C:\Users\baris\AppData\Local\Temp\ipykernel_1344\2265578016.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

In [4]:
for i in range(len(stock_daily)):
    for k in range(10,40):
        if i< len(stock_daily)-k :
            stock_daily.loc[i,"T-"+str(k)+"_HLOC_AV"]=stock_daily.loc[i+k,"HLOC_AV"]
            stock_daily.loc[i,"T-"+str(k)+"_Volume"]=stock_daily.loc[i+k,"Volume"]
            if k!= 10:
                stock_daily.loc[i,"T-"+str(k)+"_PChange"]=(stock_daily.loc[i+k,"HLOC_AV"]-stock_daily.loc[i+10,"HLOC_AV"])/(stock_daily.loc[i+10,"HLOC_AV"])
            if k%3==0:
                stock_daily.loc[i,str(k-9)+"MA"]=sum([stock_daily.loc[i+j,"HLOC_AV"] for j in range(10,k+1)])/(k-9)

obv_sum=0
for i in range(len(stock_daily)-11,-1,-1):
    if stock_daily.loc[i+10,"Close"]>stock_daily.loc[i+10,"Open"]:
        obv_sum+=stock_daily.loc[i+10,"Volume"]
    else:
        obv_sum-=stock_daily.loc[i+10,"Volume"]
    stock_daily.loc[i,"OnBalanceVolume"]=obv_sum/100000


for i in range(len(stock_daily)):
    if i < len(stock_daily)-23:
        low_14=list(stock_daily.loc[i+10:i+23,"Low"])
        high_14=list(stock_daily.loc[i+10:i+23,"High"])
        recent_close=stock_daily.loc[i+10,"Close"]
        stoch_osc=(recent_close-min(low_14))/(max(high_14)-min(low_14))
        stock_daily.loc[i,"StochasticOsc"]=stoch_osc

for k in range(3,30,3):
    first_ema=sum(stock_daily.loc[len(stock_daily)-k:len(stock_daily)-1,"Close"])/k
    stock_daily.loc[len(stock_daily)-k-10,str(k)+"EMA"]=first_ema
    for i in range(len(stock_daily)-k-10-1,-1,-1):
        previous_ema=stock_daily.loc[i+1,str(k)+"EMA"]
        multiplier=2/(k+1)
        current_ema=stock_daily.loc[i+10,"Close"]*multiplier + previous_ema*(1-multiplier)
        stock_daily.loc[i,str(k)+"EMA"]=current_ema

for i in range(len(stock_daily)-37):
    stock_daily.loc[i,"MACD"]=stock_daily.loc[i,"12EMA"]-stock_daily.loc[i,"27EMA"]

for i in range(len(stock_daily)-24):
    period_info=stock_daily.loc[i+10:i+24].reset_index(drop=True)
    positives=[]
    negatives=[]
    for index,row in period_info.iterrows():
        if index < len(period_info)-1:
            if row["Close"]>period_info["Close"][index+1]:
                positives.append(row["Close"]-period_info.loc[index+1,"Close"])
                negatives.append(0)
            elif row["Close"]<=period_info["Close"][index+1]:
                negatives.append(abs(row["Close"]-period_info.loc[index+1,"Close"]))
                positives.append(0)

    positives_avg=sum(positives)/(len(period_info)-1)
    negatives_avg=sum(negatives)/(len(period_info)-1)
    rsi=100 - ( 100 / ( 1 + ( ((positives_avg*13)+positives[0]) / ((negatives_avg*13)+negatives[0]) ) ) )
    stock_daily.loc[i,"RSI"]=rsi

for i in range(len(stock_daily)-30):
    last_21=list(stock_daily.loc[i+10:i+30,"HLOC_AV"])
    last_21_stdev=statistics.stdev(last_21)
    bolu=stock_daily.loc[i,"21MA"]+ 2*last_21_stdev
    boll=stock_daily.loc[i,"21MA"]+ 2*last_21_stdev
    stock_daily.loc[i,"BollingerU"]=bolu
    stock_daily.loc[i,"BollingerL"]=boll

for k in range(3,22,3):
    for i in range(len(stock_daily)-10-k):
        last_k=list(stock_daily.loc[i+10:i+10+k,"HLOC_AV"])
        last_k_stdev=statistics.stdev(last_k)
        stock_daily.loc[i,str(k+1)+"_StandardDev"]=last_k_stdev

# assert len(stock_daily.columns)<98,"There are too many columns. Might have duplicates."
stock_daily.iloc[-40:]

,Date,Open,High,Low,Close,Volume,AdjustedClose,HLOC_AV,T-10_HLOC_AV,T-10_Volume,...,13_StandardDev,16_StandardDev,19_StandardDev,22_StandardDev,RSI,OnBalanceVolume,StochasticOsc,BollingerU,BollingerL,MACD
9145,1986-05-08,31.9968,31.9968,31.5072,31.9968,3542400,0.0697,31.8744,31.1304,62236825,...,0.949516,1.016827,1.05778,1.143086,81.971244,-22268.73497,0.908668,30.602655,30.602655,0.841089
9146,1986-05-07,31.7536,31.9968,31.2480,31.7536,5155200,0.0692,31.6880,28.812,15609599,...,0.832786,0.84532,0.877354,1.05989,61.870402,-21646.36672,0.605119,30.195793,30.195793,0.65036
9147,1986-05-06,31.7536,32.2560,31.7536,31.7536,9734399,0.0692,31.8792,28.8808,15551999,...,0.871566,0.897676,0.921245,1.077663,56.281364,-21490.27073,0.572212,30.248397,30.248397,0.700616
9148,1986-05-05,31.5072,31.7536,31.5072,31.5072,3254400,0.0687,31.5688,29.1256,22924799,...,0.891527,0.925491,1.014849,1.064497,69.910941,-21334.75074,0.712066,30.181532,30.181532,None
9149,1986-05-02,31.7536,32.2560,31.5072,31.7536,20246399,0.0692,31.8176,29.4384,21628799,...,0.919315,0.920896,1.086963,1.038038,55.164292,-21105.50275,0.712066,30.053522,30.053522,None
9150,1986-05-01,31.7536,32.2560,31.2480,31.7536,54345597,0.0692,31.7528,30.1912,22003199,...,0.923285,0.9142,1.05441,1.009233,78.964617,-20889.21476,1.0,29.900323,29.900323,None
9151,1986-04-30,32.2560,33.2512,31.5072,32.2560,30038398,0.0703,32.3176,29.752,31910398,...,0.757203,0.845243,0.892794,0.93959,82.703158,-20669.18277,0.92596,29.616078,29.616078,None
9152,1986-04-29,33.0048,34.0000,32.7456,33.0048,30326398,0.0719,33.1888,28.7496,9302399,...,0.579808,0.798416,0.767203,0.87353,77.667845,-20350.07879,0.917735,29.53853,29.53853,None
9153,1986-04-28,34.0000,34.2432,33.4944,34.0000,28886398,0.0741,33.9344,29.0024,12153602,...,0.571719,0.762361,0.783382,0.850668,80.846699,-20257.0548,0.929616,29.55911,29.55911,None
9154,1986-04-25,33.7536,35.1232,31.9968,33.7536,85449595,0.0736,33.6568,28.816,17222399,...,0.590405,0.65873,0.82984,None,70.993915,-20135.51878,0.860146,29.4125,29.4125,None


In [5]:
stock_daily.to_csv(f"ProcessedData/{ticker_name}_Daily_Processed.csv", index=False)

In [6]:
stock_daily.iloc[-10:]

,Date,Open,High,Low,Close,Volume,AdjustedClose,HLOC_AV,T-10_HLOC_AV,T-10_Volume,...,13_StandardDev,16_StandardDev,19_StandardDev,22_StandardDev,RSI,OnBalanceVolume,StochasticOsc,BollingerU,BollingerL,MACD
9175,1986-03-26,27.2448,27.5040,26.2528,27.2448,22751999,0.0594,27.0616,None,None,...,None,None,None,None,None,None,None,None,None,None
9176,1986-03-25,26.4960,26.4960,25.7472,26.4960,32083198,0.0578,26.3088,None,None,...,None,None,None,None,None,None,None,None,None,None
9177,1986-03-24,26.0064,26.7552,25.7472,26.0064,65289596,0.0567,26.1288,None,None,...,None,None,None,None,None,None,None,None,None,None
9178,1986-03-21,26.7552,27.9936,26.2528,26.7552,59990396,0.0583,26.9392,None,None,...,None,None,None,None,None,None,None,None,None,None
9179,1986-03-20,27.5040,28.2528,27.2448,27.5040,58435196,0.0599,27.6264,None,None,...,None,None,None,None,None,None,None,None,None,None
9180,1986-03-19,28.2528,29.0016,27.9936,28.2528,47894397,0.0616,28.3752,None,None,...,None,None,None,None,None,None,None,None,None,None
9181,1986-03-18,28.7584,29.7504,28.4992,28.7584,66470396,0.0627,28.9416,None,None,...,None,None,None,None,None,None,None,None,None,None
9182,1986-03-17,29.5072,29.7504,29.0016,29.5072,133171192,0.0643,29.4416,None,None,...,None,None,None,None,None,None,None,None,None,None
9183,1986-03-14,29.0016,29.5072,27.9936,29.0016,308159981,0.0632,28.8760,None,None,...,None,None,None,None,None,None,None,None,None,None
9184,1986-03-13,27.9936,29.5072,25.5040,27.9936,1031788737,0.0610,27.7496,None,None,...,None,None,None,None,None,None,None,None,None,None
